Manon LEMAIRE - 12/2022

# **FINAL PROJECT MACHINE LEARNING**

## Text classification: goemotions 
--> Guess the sentiment(s) of the given text passage, among 28 possible labels

# **Importing libraries**

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.stem import WordNetLemmatizer
from tqdm.auto import tqdm

# **Downloading Data**

In [ ]:
train = !wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
test = !wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
evaluate = !wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

In [ ]:
train = pd.read_csv("https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv")
test = pd.read_csv("https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv")
evaluate = pd.read_csv("https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv")

# **Dropping unnecessary columns**

In [ ]:
train = train.drop(['id','author','subreddit', 'link_id', 'parent_id', 'created_utc','rater_id','example_very_unclear'], axis = 1)
test =test.drop(['id','author','subreddit', 'link_id', 'parent_id', 'created_utc','rater_id','example_very_unclear'], axis = 1)
evaluate =evaluate.drop(['id','author','subreddit', 'link_id', 'parent_id', 'created_utc','rater_id','example_very_unclear'], axis = 1)

In [ ]:
train.head()

# **Preprocessing**


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
words = set(nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()

def preprocess(sentence):
  sentence = nltk.tokenize.word_tokenize(sentence) # tokenizing
  sentence = [i.lower() for i in sentence if i not in string.punctuation] # removing punctuation
  sentence = [i for i in sentence if not i.isdigit()] # removing numbers
  sentence = [i for i in sentence if i not in stopwords] # removing stopwords
  sentence = (lemmatizer.lemmatize(w) for w in sentence) # lemmatizing 
  sentence_clean = ' '.join(w for w in sentence if w.lower() in words or not w.isalpha()) # joining the tokens

  return sentence_clean

In [ ]:
before = train['text'][:3]
after = train['text'][:3].apply(preprocess)
print(f'Before: {before.values}\n\nAfter: {after} ')

In [ ]:
tqdm.pandas() # adding a loading bar on the apply process
train['text'] = train.text.progress_apply(preprocess)
test['text'] = test.text.progress_apply(preprocess)

In [ ]:
labels = train.drop(['text'], axis = 1).values
labels_test = test.drop(['text'], axis = 1).values
print(labels.shape, labels_test.shape)